Mount Google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Predicting Sepsis patient with Med-BERT

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/greenDevil/진짜 쓸 거')
### Required Packages
from termcolor import colored
import math
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from datetime import datetime
import pickle as pkl
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch import optim
import tqdm
import time
import transformers
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import roc_curve
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
%matplotlib inline
use_cuda = torch.cuda.is_available()
import transformers
from transformers import BertForSequenceClassification, BertConfig
import copy
from sklearn.metrics import average_precision_score
import medtent # 우리가 제안하는 SFDA 기법인 Med-TENT

#### Load Data from pickled list

The pickled list is a list of lists where each sublist represent a patient record that looks like
[pt_id,label, seq_list , segment_list ]


In [3]:
train_f=pkl.load( open('/content/drive/MyDrive/greenDevil/진짜 쓸 거/labeled_data_v2/my_bertft.train_73_sepsis.pkl', 'rb'), encoding='bytes')
valid_f=pkl.load( open('/content/drive/MyDrive/greenDevil/진짜 쓸 거/labeled_data_v2/my_bertft.valid_73_sepsis.pkl', 'rb'), encoding='bytes')
test_f=pkl.load( open('/content/drive/MyDrive/greenDevil/진짜 쓸 거/labeled_data_v2/my_bertft.test_73_sepsis.pkl', 'rb'), encoding='bytes')
test_f2=pkl.load( open('/content/drive/MyDrive/greenDevil/진짜 쓸 거/labeled_data_v2/my_bertft.test_175_sepsis.pkl', 'rb'), encoding='bytes')

In [4]:
train_f[0] # 데이터 확인

('002-24088', 0, [174], [1])

In [5]:
# labeling 비율 확인
train_labels = [ex[1] for ex in train_f]
valid_labels = [ex[1] for ex in valid_f]
test_labels = [ex[1] for ex in test_f]
test2_labels = [ex[1] for ex in test_f2]
print("Train:", np.unique(train_labels, return_counts=True))
print("Valid:", np.unique(valid_labels, return_counts=True))
print("Test1 :", np.unique(test_labels, return_counts=True))
print("Test2:", np.unique(test2_labels, return_counts=True))

Train: (array([0, 1]), array([1582,  296]))
Valid: (array([0, 1]), array([232,  36]))
Test1 : (array([0, 1]), array([456,  80]))
Test2: (array([0, 1]), array([53, 11]))


In [6]:
### Below are key functions for  Data prepartion ,formating input data into features, and model defintion

class PaddingInputExample(object):
  """Fake example so the num input examples is a multiple of the batch size.

  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.

  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id,
               is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id
    self.is_real_example = is_real_example



def convert_EHRexamples_to_features(examples,max_seq_length):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    features = []
    for (ex_index, example) in enumerate(examples):
        feature = convert_singleEHR_example(ex_index, example, max_seq_length)
        features.append(feature)
    return features

### EHR version

def convert_singleEHR_example(ex_index, example, max_seq_length):
    if isinstance(example, PaddingInputExample):
        return InputFeatures(
        input_ids=[0] * max_seq_length,
        input_mask=[0] * max_seq_length,
        segment_ids=[0] * max_seq_length,
        label_id=0,
        is_real_example=False)

    input_ids=example[2]
    segment_ids=example[3]
    label_id=example[1]


  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
    input_mask = [1] * len(input_ids)


  # LR 5/13 Left Truncate longer sequence
    while len(input_ids) > max_seq_length:
        input_ids= input_ids[-max_seq_length:]
        input_mask= input_mask[-max_seq_length:]
        segment_ids= segment_ids[-max_seq_length:]



  # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length


    feature =[input_ids,input_mask,segment_ids,label_id,True]
    return feature

In [7]:
class BERTdataEHR(Dataset):
    def __init__(self, Features):

        self.data= Features


    def __getitem__(self, idx, seeDescription = False):

        sample = self.data[idx]

        return sample

    def __len__(self):
        return len(self.data)


#customized parts for EHRdataloader
def my_collate(batch):
        all_input_ids = []
        all_input_mask = []
        all_segment_ids = []
        all_label_ids = []

        for feature in batch:
            all_input_ids.append(feature[0])
            all_input_mask.append(feature[1])
            all_segment_ids.append(feature[2])
            all_label_ids.append(feature[3])
        return [all_input_ids, all_input_mask,all_segment_ids,all_label_ids]


class BERTdataEHRloader(DataLoader):
    def __init__(self, dataset, batch_size=128, shuffle=False, sampler=None, batch_sampler=None,
                 num_workers=0, collate_fn=my_collate, pin_memory=False, drop_last=False,
                 timeout=0, worker_init_fn=None):
        DataLoader.__init__(self, dataset, batch_size=batch_size, shuffle=False, sampler=None, batch_sampler=None,
                 num_workers=0, collate_fn=my_collate, pin_memory=False, drop_last=False,
                 timeout=0, worker_init_fn=None)
        self.collate_fn = collate_fn

Convert tensorflow checkpoint to pytorch

In [8]:
!python /content/drive/MyDrive/greenDevil/진짜\ 쓸\ 거/convert_bert_original_tf_checkpoint_to_pytorch.py \
  --tf_checkpoint_path=/content/drive/MyDrive/greenDevil/진짜\ 쓸\ 거/hospital73_pretraining_v2/model.ckpt-1.index \
  --bert_config_file=/content/drive/MyDrive/greenDevil/진짜\ 쓸\ 거/hospital73_pretraining_v2/config.json \
  --pytorch_dump_path=/content/converted_pytorch_model.bin

2025-11-27 08:22:51.716673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764231771.749314    4956 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764231771.759558    4956 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764231771.784197    4956 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764231771.784235    4956 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764231771.784244    4956 computation_placer.cc:177] computation placer alr

##### Model Definition

In [9]:
class EHR_BERT_LR(nn.Module):
    def __init__(self, input_size,embed_dim, hidden_size, n_layers=1,dropout_r=0.1,cell_type='LSTM',bi=False ,time=False, preTrainEmb=''):
        super(EHR_BERT_LR, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embed_dim = embed_dim
        self.dropout_r = dropout_r
        self.cell_type = cell_type
        self.preTrainEmb=preTrainEmb
        self.time=time

        if bi: self.bi=2
        else: self.bi=1

        config = BertConfig.from_json_file("/content/drive/MyDrive/greenDevil/진짜 쓸 거/hospital73_pretraining_v2/config.json")
        self.PreBERTmodel = BertForSequenceClassification(config)
        self.PreBERTmodel.load_state_dict(torch.load("/content/converted_pytorch_model.bin", map_location='cpu'), strict=False)

        # self.PreBERTmodel=BertForSequenceClassification.from_pretrained("pretrained_py_models/45M_chk")
        if use_cuda:
           self.PreBERTmodel.cuda()
        input_size=self.PreBERTmodel.bert.config.vocab_size
        self.in_size= self.PreBERTmodel.bert.config.hidden_size

        self.dropout = nn.Dropout(p=self.dropout_r)
        self.out = nn.Linear(self.in_size,1)
        self.sigmoid = nn.Sigmoid()
        self.softmax=nn.Softmax()
        if use_cuda:
            self.flt_typ=torch.cuda.FloatTensor
            self.lnt_typ=torch.cuda.LongTensor
        else:
            self.lnt_typ=torch.LongTensor
            self.flt_typ=torch.FloatTensor

    def forward(self, sequence):
        token_t=torch.from_numpy(np.asarray(sequence[0],dtype=int)).type(self.lnt_typ)
        seg_t=torch.from_numpy(np.asarray(sequence[2],dtype=int)).type(self.lnt_typ)
        Label_t=torch.from_numpy(np.asarray(sequence[3],dtype=int)).type(self.lnt_typ)
        Bert_out=self.PreBERTmodel.bert(input_ids=token_t, attention_mask=torch.from_numpy(np.asarray(sequence[1],dtype=int)).type(self.lnt_typ),
                                    token_type_ids=seg_t)
        output=self.sigmoid(self.out(Bert_out[1]))

        return output.squeeze(),Label_t.type(self.flt_typ)

In [10]:
def unpack_from_mycollate(batch, use_cuda=False, num_labels=1):
    """
    my_collate가 반환한 형태:
      [all_input_ids, all_input_mask, all_segment_ids, all_label_ids]
    그런데 외부에서 한 번 더 래핑되어 [[...]] 형태로 올 수 있어 그리면 한 겹 벗겨줌
    반환: (input_ids[ B,L ] Long, attention_mask[ B,L ] Long,
           token_type_ids[ B,L ] Long, labels[ B ] Float(Long))
    """
    # 한 겹 더 싸여 있으면 풀기
    if isinstance(batch, (list, tuple)) and len(batch) == 1 and isinstance(batch[0], (list, tuple)):
        batch = batch[0]

    ids_li, mask_li, seg_li, lab_li = batch  # 4개 리스트
    # 리스트/넘파이 → [B,L] LongTensor
    input_ids      = torch.as_tensor(np.asarray(ids_li),  dtype=torch.long)
    attention_mask = torch.as_tensor(np.asarray(mask_li), dtype=torch.long)
    token_type_ids = torch.as_tensor(np.asarray(seg_li),  dtype=torch.long)

    # 라벨 → [B]
    labels = torch.as_tensor(np.asarray(lab_li))
    labels = labels.float() if num_labels == 1 else labels.long()

    if use_cuda:
        input_ids = input_ids.cuda(non_blocking=True)
        attention_mask = attention_mask.cuda(non_blocking=True)
        token_type_ids = token_type_ids.cuda(non_blocking=True)
        labels = labels.cuda(non_blocking=True)

    return input_ids, attention_mask, token_type_ids, labels

In [11]:
def timeSince(since):
   now = time.time()
   s = now - since
   m = math.floor(s / 60)
   s -= m * 60
   return '%dm %ds' % (m, s)

def trainsample(sample, model, optimizer, criterion = nn.BCELoss()):
    model.train()
    model.zero_grad()
    output,label_tensor = model(sample)
    #label_tensor = smooth_labels(label_tensor, 0.1)
    loss = criterion(output, label_tensor)
    loss.backward()
    optimizer.step()
    return output, loss.item()


#train with loaders

def trainbatches(mbs_list, model, optimizer, shuffle = True):
    current_loss = 0
    all_losses =[]
    plot_every = 5
    n_iter = 0
    if shuffle:
        random.shuffle(mbs_list)
    for i,batch in enumerate(mbs_list):
        output, loss = trainsample(batch, model, optimizer, criterion = nn.BCELoss())
        current_loss += loss
        n_iter +=1

        if n_iter % plot_every == 0:
            all_losses.append(current_loss/plot_every)
            current_loss = 0
    return current_loss, all_losses


def calculate_auc(model, mbs_list, shuffle = False):
    model.eval()
    y_real =[]
    y_hat= []
    if shuffle:
        random.shuffle(mbs_list)
    for i,batch in enumerate(mbs_list):
        output,label_tensor = model(batch)
        y_hat.extend(output.cpu().data.view(-1).numpy())
        y_real.extend(label_tensor.cpu().data.view(-1).numpy())
    auc = roc_auc_score(y_real, y_hat)
    ap = average_precision_score(y_real, y_hat)

    return auc, ap, y_real, y_hat


In [12]:
def epochs_run(epochs, train, valid, test1,test2, model, optimizer, shuffle = True,  patience = 20, output_dir = '../models/', model_prefix = 'dhf.train', model_customed= ''):
    bestValidAuc = 0.0
    bestTestAuc1 = 0.0
    bestTestAuc2 = 0.0
    bestValidEpoch = 0
    #header = 'BestValidAUC|TestAUC|atEpoch'
    #logFile = output_dir + model_prefix + model_customed +'EHRmodel.log'
    #print2file(header, logFile)
    #writer = SummaryWriter(output_dir+'/tsb_runs/') ## LR added 9/27 for tensorboard integration
    for ep in range(epochs):
        print (ep)
        start = time.time()
        current_loss, train_loss = trainbatches(mbs_list = train, model= model, optimizer = optimizer)
        train_time = timeSince(start)
        #epoch_loss.append(train_loss)
        avg_loss = np.mean(train_loss)
        #writer.add_scalar('Loss/train', avg_loss, ep) ## LR added 9/27
        valid_start = time.time()
        train_auc, train_ap, _, _ = calculate_auc(model = model, mbs_list = train, shuffle = False)
        valid_auc, valid_ap, _, _ = calculate_auc(model = model, mbs_list = valid, shuffle = False)
        valid_time = timeSince(valid_start)
        #writer.add_scalar('train_auc', train_auc, ep) ## LR added 9/27
        #writer.add_scalar('valid_auc', valid_auc, ep) ## LR added 9/27
        print(colored('\n Epoch (%s): Train_auc (%s), Valid_auc (%s) ,Training Average_loss (%s), Train_time (%s), Eval_time (%s)'%(ep, train_auc, valid_auc , avg_loss,train_time, valid_time), 'green'))
        print(colored('\n Epoch (%s): Train_auprc (%s), Valid_auprc (%s) ,Training Average_loss (%s), Train_time (%s), Eval_time (%s)'%(ep, train_ap, valid_ap , avg_loss,train_time, valid_time), 'green'))
        if valid_auc > bestValidAuc:
              bestValidAuc = valid_auc
              bestValidEpoch = ep
              best_model= model
              bestTrainAuc = train_auc
              bestTrainAp = train_ap
              if test:
                      testeval_start = time.time()
                      bestTestAuc1, bestTestAp1, _, _ = calculate_auc(model = best_model, mbs_list = test1,  shuffle = False)
                      bestTestAuc2, bestTestAp2, _, _ = calculate_auc(model = best_model, mbs_list = test2,  shuffle = False)

                        #writer.add_scalar('test_auc', valid_auc, ep) ## LR added 9/27
                      print(colored('\n Test_AUC1 (%s) ,Test_AUC2 (%s) , Test_eval_time (%s) '%(bestTestAuc1,bestTestAuc2, timeSince(testeval_start)), 'yellow'))
                      print(colored('\n Test_AUPRC1 (%s) ,Test_AUPRC2 (%s) , Test_eval_time (%s) '%(bestTestAp1,bestTestAp2, timeSince(testeval_start)), 'yellow'))
                      #print(best_model,model) ## to verify that the hyperparameters already impacting the model definition
                      #print(optimizer)
        if ep - bestValidEpoch > patience:
              break

    #writer.close()
    #if not os.path.exists(output_dir):
    #    os.makedirs(output_dir)
    ###save model & parameters
    #torch.save(best_model, output_dir + model_prefix + model_customed + 'EHRmodel.pth')
    torch.save(best_model.state_dict(), output_dir + model_prefix + model_customed + 'EHRmodel.st')

    if test:
        print(colored('BestValidAuc %f has a TestAuc of %f at epoch %d ' % (bestValidAuc, bestTestAuc1, bestValidEpoch),'green'))
        return bestTrainAuc, bestTrainAp, bestValidAuc, bestTestAuc1, bestTestAp1, bestTestAuc2, bestTestAp2, bestValidEpoch
    else:
        print(colored('BestValidAuc %f at epoch %d ' % (bestValidAuc,  bestValidEpoch),'green'))
        print('No Test Accuracy')

    print(colored('Details see ../models/%sEHRmodel.log' %(model_prefix + model_customed),'green'))

In [13]:
# Med-TENT version
def new_epochs_run(make_model_fn, checkpoint_path, loader, lr, use_cuda=False, num_labels=1):
    """
    make_model_fn: 동일 구조 모델 생성 함수
    checkpoint_path: fine-tuned best state_dict 경로
    loader: test2 DataLoader (my_collate 기반)
    num_labels: 1(시그모이드) 또는 2(소프트맥스)
    """
    def disable_dropout(m):
      for name, child in m.named_children():
        if isinstance(child, torch.nn.Dropout):
            setattr(m, name, torch.nn.Identity())
        else:
            disable_dropout(child)

    # 동일 가중치 모델 다시 로드 (적응 누적 방지)
    ehr = make_model_fn()
    state = torch.load(checkpoint_path, map_location=("cuda" if use_cuda else "cpu"))
    ehr.load_state_dict(state)
    if use_cuda:
        ehr.cuda()

    # Med-TENT 셋업
    base = ehr.PreBERTmodel
    disable_dropout(base)
    base = medtent.configure_model(base)                # LN만 requires_grad=True
    ln_params, ln_names = medtent.collect_params(base, top_k=4, include_classifier_bias=False)  # bias update
    # optimizer = optim.Adam(ln_params, lr=lr)
    optimizer = optim.SGD(
      ln_params,
      lr=1e-3,
      momentum=0.9,
      nesterov=True,
      weight_decay=0.0
    )

    model2 = medtent.MedTent(base, optimizer, steps=1, episodic=True)

    y_true, y_hat = [], []
    base.train()  # test-time에도 train 모드 유지

    for batch in loader:
        # my_collate → [all_ids, all_mask, all_segs, all_labels] 형태를 텐서로 정규화
        input_ids, attention_mask, token_type_ids, labels = unpack_from_mycollate(
            batch, use_cuda=use_cuda, num_labels=2
        )
        batch_dict = {"input_ids": input_ids, "attention_mask": attention_mask}

        if token_type_ids is not None:
            token_type_ids = torch.zeros_like(input_ids)

        outputs = model2(batch_dict)
        logits = outputs.logits.detach().cpu().numpy()

        if logits.ndim == 2 and logits.shape[1] == 1:
            #sigmoid
            probs = 1.0 / (1.0 + np.exp(-logits[:, 0]))
        else:
            #softmax
            e = np.exp(logits - logits.max(axis=1, keepdims=True))
            probs = (e / e.sum(axis=1, keepdims=True))[:, 1]

        y_hat.extend(probs.tolist())
        y_true.extend(labels.detach().cpu().numpy().reshape(-1).tolist())

    auc = roc_auc_score(y_true, y_hat)
    ap = average_precision_score(y_true, y_hat)
    return auc, ap, np.array(y_true), np.array(y_hat)

Main

In [14]:
MAX_SEQ_LENGTH = 128
BATCH_SIZE = 100
LEARNING_RATE = 5e-5

results=[]

#### Data Preparation
train_features = convert_EHRexamples_to_features(train_f, MAX_SEQ_LENGTH)
test_features = convert_EHRexamples_to_features(test_f, MAX_SEQ_LENGTH)
test_features2 = convert_EHRexamples_to_features(test_f2, MAX_SEQ_LENGTH)
valid_features = convert_EHRexamples_to_features(valid_f, MAX_SEQ_LENGTH)
train = BERTdataEHR(train_features)
test = BERTdataEHR(test_features)
test2 = BERTdataEHR(test_features2)
valid = BERTdataEHR(valid_features)
print (' creating the list of training minibatches')
train_mbs = list(BERTdataEHRloader(train, batch_size = BATCH_SIZE))
print (' creating the list of test minibatches')
test_mbs = list(BERTdataEHRloader(test, batch_size = BATCH_SIZE))
print (' creating the list of test2 minibatches')
test_mbs2 = list(BERTdataEHRloader(test2, batch_size = BATCH_SIZE))
print (' creating the list of valid minibatches')
valid_mbs = list(BERTdataEHRloader(valid, batch_size = BATCH_SIZE))

for run in range(10):### to average the results on 10 runs
    for model_type in ['Bert only']:
            ehr_model = EHR_BERT_LR(input_size= 90000, embed_dim=192, hidden_size=192)
            if use_cuda:
                ehr_model.cuda()
            optimizer = optim.Adam(ehr_model.parameters(), lr=LEARNING_RATE)
            out_dir_name='test_LR_Bert_BiGRU_FC'#+ str(i)
            trauc, trap, vauc, testauc1, testap1, testauc2, testap2, bep = epochs_run(10,train = train_mbs,
                                  valid = valid_mbs,
                                  test1 = test_mbs,test2=test_mbs2,
                                  model = ehr_model,
                                  optimizer = optimizer,
                                  shuffle = False,
                                  patience = 20,
                                  output_dir = out_dir_name,
                                  model_prefix = 'first_run')
            results.append([model_type,run,len(train_features),len(test_features),len(valid_features),trauc, trap, vauc,testauc1,testap1, testauc2, testap2, bep])


 creating the list of training minibatches
 creating the list of test minibatches
 creating the list of test2 minibatches
 creating the list of valid minibatches
0

 Epoch (0): Train_auc (0.6628497966993543), Valid_auc (0.6695402298850575) ,Training Average_loss (0.5976126174132029), Train_time (0m 2s), Eval_time (0m 0s)

 Epoch (0): Train_auprc (0.30286914670872195), Valid_auprc (0.2156098669865085) ,Training Average_loss (0.5976126174132029), Train_time (0m 2s), Eval_time (0m 0s)

 Test_AUC1 (0.6449835526315789) ,Test_AUC2 (0.5274442538593482) , Test_eval_time (0m 0s) 

 Test_AUPRC1 (0.2689663431132175) ,Test_AUPRC2 (0.2290285630630952) , Test_eval_time (0m 0s) 
1

 Epoch (1): Train_auc (0.7518707076229201), Valid_auc (0.7533524904214559) ,Training Average_loss (0.432432222366333), Train_time (0m 1s), Eval_time (0m 1s)

 Epoch (1): Train_auprc (0.375102465382131), Valid_auprc (0.3075950536456683) ,Training Average_loss (0.432432222366333), Train_time (0m 1s), Eval_time (0m 1s)

 Test

Result & Evaluation

In [15]:
df=pd.DataFrame(results)
df.columns=['Model','Run','Train_size','Test_size','Valid_size','Train_AUC','Train_AUPRC','Valid_AUC','Test_AUC1','Test_AUPRC1','Test_AUC2','Test_AUPRC2','Best_Epoch']

In [16]:
df

,Model,Run,Train_size,Test_size,Valid_size,Train_AUC,Train_AUPRC,Valid_AUC,Test_AUC1,Test_AUPRC1,Test_AUC2,Test_AUPRC2,Best_Epoch
0,Bert only,0,1878,536,268,0.893474,0.665045,0.878592,0.815296,0.445179,0.544597,0.222422,6
1,Bert only,1,1878,536,268,0.902740,0.687587,0.877514,0.818298,0.460141,0.563465,0.218143,6
2,Bert only,2,1878,536,268,0.920661,0.745026,0.879071,0.822684,0.471045,0.510292,0.194588,8
3,Bert only,3,1878,536,268,0.897197,0.680077,0.877155,0.818969,0.460184,0.580617,0.223340,6
4,Bert only,4,1878,536,268,0.903849,0.690538,0.881106,0.818325,0.461293,0.560034,0.229029,6
5,Bert only,5,1878,536,268,0.900054,0.678271,0.879071,0.818353,0.465077,0.561750,0.210682,6
6,Bert only,6,1878,536,268,0.897758,0.682560,0.879550,0.816338,0.463969,0.541166,0.210473,6
7,Bert only,7,1878,536,268,0.898691,0.676306,0.882543,0.820792,0.461154,0.560034,0.226983,6
8,Bert only,8,1878,536,268,0.900795,0.686088,0.881226,0.816913,0.469725,0.551458,0.215715,6
9,Bert only,9,1878,536,268,0.907989,0.703973,0.879071,0.821491,0.463839,0.563465,0.208787,7


In [17]:
desc1=df[['Model','Train_size','Train_AUC','Train_AUPRC']].groupby(['Model','Train_size']).describe()
desc2=df[['Model','Train_size','Test_AUC1','Test_AUPRC1']].groupby(['Model','Train_size']).describe()
desc3=df[['Model','Train_size','Test_AUC2','Test_AUPRC2']].groupby(['Model','Train_size']).describe()

print(f"[Source Domain]] Train AUROC: {desc1[('Train_AUC', 'mean')].iloc[0]:.3f}")
print(f"[Source Domain] Test1 AUROC: {desc2[('Test_AUC1', 'mean')].iloc[0]:.3f}")
print(f"[Target Domain(label X)] Test2 AUROC: {desc3[('Test_AUC2', 'mean')].iloc[0]:.3f}")
print("-----------------------------------")
print(f"[Source Domain]] Train AUPRC: {desc1[('Train_AUPRC', 'mean')].iloc[0]:.3f}")
print(f"[Source Domain] Test1 AUPRC: {desc2[('Test_AUPRC1', 'mean')].iloc[0]:.3f}")
print(f"[Target Domain(label X)] Test2 AUPRC: {desc3[('Test_AUPRC2', 'mean')].iloc[0]:.3f}")

[Source Domain]] Train AUROC: 0.902
[Source Domain] Test1 AUROC: 0.819
[Target Domain(label X)] Test2 AUROC: 0.554
-----------------------------------
[Source Domain]] Train AUPRC: 0.690
[Source Domain] Test1 AUPRC: 0.462
[Target Domain(label X)] Test2 AUPRC: 0.216
